In [15]:
AGENT_NAME = "s020-streaming"

In [16]:
from agentex import Agentex

client = Agentex(api_key="random", base_url="http://localhost:5003")

In [17]:
# # (Optional) Create a new task. If you don't create a new task, each message will be sent to a new task. The server will create the task for you.

# import uuid

# TASK_ID = str(uuid.uuid4())[:8]

# rpc_response = client.agents.rpc_by_name(
#     agent_name=AGENT_NAME,
#     method="task/create",
#     params={
#         "name": f"{TASK_ID}-task",
#         "params": {}
#     }
# )

# task = rpc_response.result
# print(task)

In [18]:
# Test non streaming response
from agentex.types import TaskMessage, TextContent

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="message/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": False
    }
)

# # Extract and print just the text content from the response
# # The response is expected to be a dict with a "result" key containing a list of message dicts
if rpc_response and rpc_response.result:
    for message in rpc_response.result:
        if isinstance(message, TaskMessage):
            content = message.content
            if isinstance(content, TextContent):
                text = content.content
                print(text)
        else:
            print("No text content found in response.")


Hello! I can assist you with a variety of tasks, including:

1. **Answering Questions**: I can provide information on a wide range of topics, including science, history, technology, and more.
2. **Explaining Concepts**: If you need help understanding a concept or topic, I can provide explanations and clarifications.
3. **Writing Assistance**: I can help with writing tasks, such as drafting emails, essays, or creative pieces.
4. **Language Support**: I can assist with grammar, vocabulary, and writing in different languages.
5. **Providing Recommendations**: Whether it's books, movies, or recipes, I can suggest options based on your preferences.
6. **Problem-Solving**: I can help you think through problems or brainstorm ideas.

If there's something specific you need help with, feel free to ask!


In [19]:
# Test streaming response
import json
from agentex.types import AgentRpcResponse
from agentex.types.agent_rpc_result import StreamTaskMessageDelta, StreamTaskMessageFull
from agentex.types.text_delta import TextDelta

with client.agents.with_streaming_response.rpc_by_name(
    agent_name=AGENT_NAME,
    method="message/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "stream": True
    }
) as response:
    for streaming_response_str in response.iter_text():
        chunk_rpc_responses = streaming_response_str.split("\n")
        for chunk_rpc_response in chunk_rpc_responses:
            if chunk_rpc_response:
                chunk_rpc_response_dict = json.loads(chunk_rpc_response)
                chunk_rpc_response = AgentRpcResponse.model_validate(chunk_rpc_response_dict)
                result = chunk_rpc_response.result
                # Print only the text deltas as they arrive or any full messages
                if isinstance(result, StreamTaskMessageDelta):
                    delta = result.delta
                    if isinstance(delta, TextDelta):
                        print(delta.text_delta, end="", flush=True)
                elif isinstance(result, StreamTaskMessageFull):
                    content = result.content
                    if isinstance(content, TextContent):
                        print(content.content)

Hello! I can help with a variety of tasks, including:

1. **Answering Questions**: I can provide information on a wide range of topics, from historical facts to scientific concepts.

2. **Providing Explanations**: If you're looking to understand a specific topic better, I can explain it in detail.

3. **Offering Writing Assistance**: I can help with writing prompts, editing text, or generating ideas for essays, articles, and more.

4. **Solving Problems**: I can assist with math problems, logic puzzles, and more.

5. **Learning Support**: I can help with study tips, summarizing information, and creating study guides.

6. **Conversational Practice**: If you want to practice a language or just have a chat, I’m here for that too!

7. **Recommendation Systems**: I can recommend books, movies, or other media based on your interests.

Feel free to ask me anything specific you need help with!